In [1]:
import sys
sys.executable

'/root/anaconda3/envs/ystorch/bin/python'

In [18]:
def getRating():
    ranking_file = '../LIRIS-ACCEDE-annotations/annotations/ACCEDEranking.txt'
    import pandas as pd
    ranking = pd.read_csv(ranking_file, delimiter='\t')
    return ranking
rating = getRating()
rating

,id,name,valenceRank,arousalRank,valenceValue,arousalValue,valenceVariance,arousalVariance
0,0,ACCEDE00000.mp4,3755,787,2.813984,1.366550,0.197940,0.144121
1,1,ACCEDE00001.mp4,6711,6428,3.347393,2.791904,0.199892,0.148621
2,2,ACCEDE00002.mp4,5043,3591,2.981398,1.840760,0.195831,0.151290
3,3,ACCEDE00003.mp4,7584,5615,3.279497,2.470491,0.174483,0.160281
4,4,ACCEDE00004.mp4,5014,4463,2.983141,2.082050,0.199350,0.162548
...,...,...,...,...,...,...,...,...
9795,9795,ACCEDE09795.mp4,8315,3236,3.429017,1.755565,0.188731,0.144128
9796,9796,ACCEDE09796.mp4,9784,1775,3.637237,1.481259,0.193705,0.118789
9797,9797,ACCEDE09797.mp4,9729,1560,3.618762,1.451211,0.172837,0.119660
9798,9798,ACCEDE09798.mp4,9605,100,3.567955,1.321688,0.183909,0.193999


In [3]:
rating[rating['name'] == 'ACCEDE09797.mp4']['valenceValue'].values[0]

3.61876162879

In [4]:
'../LIRIS-ACCEDE-data/data/ACCEDE00068.mp4'.split('/')[-1]

'ACCEDE00068.mp4'

In [5]:
batch_size=1
sequence_length=64

initial_prefetch_size=11

shuffle=False

n_iter=6

In [6]:
from __future__ import print_function
from __future__ import division
import os
import numpy as np

from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types

In [9]:
# https://docs.nvidia.com/deeplearning/sdk/dali-developer-guide/docs/supported_ops.html#nvidia.dali.ops.VideoReader
class VideoPipe(Pipeline):
    def __init__(self, batch_size, num_threads, device_id, data, shuffle):
        super(VideoPipe, self).__init__(batch_size, num_threads, device_id, seed=16)
        self.input = ops.VideoReader(device="gpu", file_list=data, sequence_length=sequence_length,
                                     shard_id=0, num_shards=1,
                                     random_shuffle=shuffle, initial_fill=initial_prefetch_size)
        self.resize = ops.Resize(device = "gpu",
                                 image_type = types.RGB,
                                 interp_type = types.INTERP_LINEAR,
                                 resize_shorter = 224
                                )


    def define_graph(self):
        inputs,labels = self.input(name="Reader")
#         inputs = self.resize(inputs)
        return (inputs, labels)

In [10]:
def getVideoList():
    data_dir = '/root/yangsen-data/LIRIS-ACCEDE-data/data/'

    import glob
    video_list = glob.glob(data_dir + '*.mp4')
    return video_list

# 统计clip帧数分布

In [13]:
frames = [torchvision.io.read_video(i)[0].shape[1] for i in getVideoList()]
frames

[400,
 720,
 720,
 576,
 360,
 1016,
 400,
 360,
 240,
 640,
 360,
 320,
 360,
 352,
 360,
 320,
 352,
 360,
 400,
 532,
 576,
 720,
 320,
 240,
 576,
 360,
 352,
 720,
 360,
 360,
 720,
 720,
 400,
 360,
 720,
 320,
 352,
 360,
 320,
 720,
 720,
 1080,
 640,
 344,
 320,
 376,
 720,
 320,
 240,
 720,
 480,
 576,
 720,
 640,
 720,
 376,
 268,
 400,
 360,
 720,
 360,
 720,
 272,
 720,
 352,
 720,
 352,
 320,
 480,
 268,
 360,
 576,
 360,
 720,
 352,
 360,
 720,
 360,
 268,
 480,
 720,
 352,
 376,
 720,
 720,
 268,
 304,
 720,
 360,
 400,
 720,
 320,
 720,
 352,
 720,
 360,
 360,
 320,
 576,
 360,
 240,
 720,
 320,
 360,
 240,
 720,
 360,
 352,
 352,
 532,
 720,
 352,
 320,
 360,
 720,
 720,
 240,
 352,
 268,
 720,
 400,
 720,
 576,
 400,
 720,
 352,
 352,
 720,
 352,
 720,
 720,
 360,
 400,
 320,
 720,
 720,
 720,
 240,
 720,
 352,
 720,
 360,
 720,
 720,
 360,
 304,
 352,
 360,
 640,
 360,
 360,
 400,
 352,
 576,
 352,
 720,
 240,
 352,
 352,
 304,
 360,
 360,
 304,
 360,
 352,
 400,
 2

In [11]:
video_list = getVideoList()
video_list

['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE09585.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE02064.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE07025.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE04830.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE04183.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE07077.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE09504.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE00215.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE03677.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE08964.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE00191.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE08046.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE00465.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE03470.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE00168.mp4',
 '/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE02609.mp4',
 '/root/

In [12]:
video_and_label = [[i, rating[rating['name'] == i.split('/')[-1]]['valenceValue'].values[0]] for i in video_list]
video_and_label

[['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE09585.mp4', 2.90281166597],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE02064.mp4', 1.63925629371],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE07025.mp4', 3.528843563],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE04830.mp4', 1.54445892272],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE04183.mp4', 2.98957009069],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE07077.mp4', 3.42072133379],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE09504.mp4',
  3.2186552994200004],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE00215.mp4', 3.29158212872],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE03677.mp4',
  3.0056137361900004],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE08964.mp4', 3.22989704254],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE00191.mp4', 3.27305620661],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE08046.mp4', 3.39707801967],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/

In [13]:
video_and_name = [[i, i.split('/')[-1]] for i in video_list]
video_and_name

[['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE09585.mp4',
  'ACCEDE09585.mp4'],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE02064.mp4',
  'ACCEDE02064.mp4'],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE07025.mp4',
  'ACCEDE07025.mp4'],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE04830.mp4',
  'ACCEDE04830.mp4'],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE04183.mp4',
  'ACCEDE04183.mp4'],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE07077.mp4',
  'ACCEDE07077.mp4'],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE09504.mp4',
  'ACCEDE09504.mp4'],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE00215.mp4',
  'ACCEDE00215.mp4'],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE03677.mp4',
  'ACCEDE03677.mp4'],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE08964.mp4',
  'ACCEDE08964.mp4'],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE00191.mp4',
  'ACCEDE00191.mp4'],
 ['/root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE08046.mp4',
  

In [14]:
file_list_path = 'file_list.txt'
with open(file_list_path, 'w') as f:
    for i in video_and_name:
        print(i[0] + ' ' + str(int(i[1][6:11])), file=f)

In [15]:
def constructToFileName(num: int) -> str:
    return f'ACCEDE{num:05}.mp4'
print(constructToFileName(1))
print(constructToFileName(12))
print(constructToFileName(12342))

ACCEDE00001.mp4
ACCEDE00012.mp4
ACCEDE12342.mp4


In [16]:
pipe = VideoPipe(batch_size=batch_size, num_threads=1, device_id=0, data=file_list_path, shuffle=shuffle)
pipe.build()
for i in range(n_iter):
    sequences_out = pipe.run()[0]
    sequences_out = sequences_out.as_cpu().as_array()
    print(sequences_out.shape)


RuntimeError: [/opt/dali/dali/operators/reader/loader/video_loader.cc:223] Could not open file /root/yangsen-data/LIRIS-ACCEDE-data/data/ACCEDE03201.mp4 because of Too many open files
Stacktrace (100 entries):
[frame 0]: /root/anaconda3/envs/ystorch/lib/python3.7/site-packages/nvidia/dali/libdali_operators.so(+0x2b56fe) [0x7fc0375146fe]
[frame 1]: /root/anaconda3/envs/ystorch/lib/python3.7/site-packages/nvidia/dali/libdali_operators.so(+0x7fc5ab) [0x7fc037a5b5ab]
[frame 2]: /root/anaconda3/envs/ystorch/lib/python3.7/site-packages/nvidia/dali/libdali_operators.so(+0x804c92) [0x7fc037a63c92]
[frame 3]: /root/anaconda3/envs/ystorch/lib/python3.7/site-packages/nvidia/dali/libdali_operators.so(+0x82b88b) [0x7fc037a8a88b]
[frame 4]: /root/anaconda3/envs/ystorch/lib/python3.7/site-packages/nvidia/dali/libdali_operators.so(+0x82cbe2) [0x7fc037a8bbe2]
[frame 5]: /root/anaconda3/envs/ystorch/lib/python3.7/site-packages/nvidia/dali/libdali_operators.so(std::_Function_handler<std::unique_ptr<dali:

In [ ]:
sequences_out = pipe.run()[0]
sequences_out = sequences_out.as_cpu().as_array()

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
def show_sequence(sequence, label):
    columns = 4
    rows = (sequence_length + 1) // (columns)
    fig = plt.figure(figsize = (32,(16 // columns) * rows))
    gs = gridspec.GridSpec(rows, columns)
    for j in range(rows*columns):
        plt.subplot(gs[j])
        plt.axis("off")
        plt.suptitle("label " + str(label[0]), fontsize=30)
        plt.imshow(sequence[j])

In [ ]:
ITER =2
for i in range(ITER):
    sequences_out = pipe.run()[0]
    sequences_out = sequences_out.as_cpu().as_array()
    show_sequence(sequences_out[0], '1')    

In [ ]:
sample0 = pipe.run()[0]
sample0.as_cpu().as_array().shape

# transfer learning

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [ ]:
# integration with pytorch
from nvidia.dali.plugin.pytorch import DALIGenericIterator

In [ ]:
pipe = VideoPipe(batch_size=batch_size, num_threads=1, device_id=0, data=file_list_path, shuffle=shuffle)
pipe.build()

In [ ]:
dali_iter = DALIGenericIterator([pipe], ['data', 'label'], pipe.epoch_size('Reader'))

In [ ]:
def new_dali_iter():
    return DALIGenericIterator([pipe], ['data', 'label'], pipe.epoch_size('Reader'))

# dali_iter[0]

In [ ]:
ITERATIONS = 5
for i, data in enumerate(dali_iter):
    if i >= ITERATIONS:
        break
    # Testing correctness of labels
    for d in data:
        image = d["data"]
        name = d['label']
        print(name.shape)
        name = torch.tensor(name, dtype=torch.float64)
        print(image.shape)
        print(name)
        affective = np.zeros((2,2), dtype=float)
        print(affective)
        for row in range(len(name)):
            for col in range(len(name[row])):
                video_index = int(name[row][col].data.cpu().numpy())
                print(video_index)
                affective[row][0] = rating[rating['name'] == constructToFileName(video_index)]['valenceValue'].values[0]
                affective[row][1] = rating[rating['name'] == constructToFileName(video_index)]['arousalValue'].values[0]
        print('affective:', affective)



In [ ]:
def uniform_rescale(x):
    return (x / 9800) * 2 - 1
def getAffective(name, column='value'):
    name = torch.tensor(name, dtype=torch.float64)
    affective = np.zeros((batch_size,2), dtype=float)
    for row in range(len(name)):
        for col in range(len(name[row])):
            video_index = int(name[row][col].data.cpu().numpy())
            if column == 'value':
                affective[row][0] = rating[rating['name'] == constructToFileName(video_index)]['valenceValue'].values[0]
                affective[row][1] = rating[rating['name'] == constructToFileName(video_index)]['arousalValue'].values[0]
            elif column == 'rank':
                affective[row][0] = uniform_rescale(rating[rating['name'] == constructToFileName(video_index)]['valenceRank'].values[0])
                affective[row][1] = uniform_rescale(rating[rating['name'] == constructToFileName(video_index)]['arousalRank'].values[0])
    return torch.tensor(affective)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    phase = 'train'
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss = 0.0
        data_set_size = 0

        # Iterate over data.
        for data in new_dali_iter():
            for d in data:
                inputs = d['data']
                inputs = inputs.view(-1, 376, 640, 3).permute(0, 3, 1, 2).to(device=device, dtype=torch.float)
                labels = getAffective(d['label'])
                labels = torch.cat(sequence_length*[labels])
                labels = labels.to(device=device, dtype=torch.float)
                
#                 print (inputs)
#                 print (labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
#                     print(outputs)
#                     print(labels)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                data_set_size += 1
                if phase == 'train':
                    scheduler.step()

        epoch_loss = running_loss / data_set_size

        print('{} Loss: {:.4f}'.format(
            phase, epoch_loss))

        best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def predict_affective(model):
    ret = {}
    with torch.no_grad():
        for data in new_dali_iter():
            for d in data:
                inputs = d['data']
                inputs = inputs.view(-1, 376, 640, 3).permute(0, 3, 1, 2).to(device=device, dtype=torch.float)
                video_name = d['label'].cpu().numpy()[0][0]
                # forward          
                outputs = model(inputs)
                ret[video_name] = outputs.cpu().numpy()
    return ret
            
results = predict_affective(model_ft)

In [ ]:
results

In [ ]:
image0 = results[48]
import matplotlib.path as mpath
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

%matplotlib inline

fig, ax = plt.subplots()
x, y = zip(*image0)
line, = ax.plot(x, y, '-', label='48')

ax.grid()
ax.axis([1,5,1,5])
ax.legend()
plt.xlabel('Arousal')
plt.ylabel('Valence')
plt.show()

In [ ]:
import matplotlib.path as mpath
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

%matplotlib inline

fig, ax = plt.subplots()
i = 0
print (f'length: {len(results)}')
for image_name in [43, 101]:
#     if i > 10:
#         break
#     i += 1
#     if i % 2 == 1:
#         continue
    x, y = zip(*results[image_name])
    line, = ax.plot(x, y, '-', label=str(image_name))

ax.grid()
# ax.axis([1,5,1,5])
ax.axis('equal')
ax.legend()
plt.xlabel('Arousal')
plt.ylabel('Valence')
plt.show()
    

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [49]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch.nn.functional as F
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.MSELoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

为什么loss不下降？

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=5)

# Movie 的 VA曲线

In [12]:
description = '/root/yangsen-data/LIRIS-ACCEDE-data/ACCEDEdescription.xml'

In [13]:
import xml.etree.ElementTree as ET
root = ET.parse(description).getroot()

In [14]:
def getAffectiveValue(name):
    return [rating[rating['name'] == name]['valenceValue'].values[0], 
            rating[rating['name'] == name]['arousalValue'].values[0]]

In [16]:
movies = {}
for media in root.findall('media'):
    idd = media.find('id')
    name = media.find('name')
    m = media.find('movie')
    if m.text not in movies:
        movies[m.text] = []
    movies[m.text].append(name.text)
movies

{'20_Mississippi.mp4': ['ACCEDE00000.mp4',
  'ACCEDE00001.mp4',
  'ACCEDE00002.mp4',
  'ACCEDE00003.mp4',
  'ACCEDE00004.mp4',
  'ACCEDE00005.mp4',
  'ACCEDE00006.mp4',
  'ACCEDE00007.mp4',
  'ACCEDE00008.mp4',
  'ACCEDE00009.mp4',
  'ACCEDE00010.mp4',
  'ACCEDE00011.mp4',
  'ACCEDE00012.mp4',
  'ACCEDE00013.mp4',
  'ACCEDE00014.mp4',
  'ACCEDE00015.mp4',
  'ACCEDE00016.mp4',
  'ACCEDE00017.mp4',
  'ACCEDE00018.mp4',
  'ACCEDE00019.mp4',
  'ACCEDE00020.mp4',
  'ACCEDE00021.mp4',
  'ACCEDE00022.mp4',
  'ACCEDE00023.mp4',
  'ACCEDE00024.mp4',
  'ACCEDE00025.mp4',
  'ACCEDE00026.mp4',
  'ACCEDE00027.mp4',
  'ACCEDE00028.mp4',
  'ACCEDE00029.mp4',
  'ACCEDE00030.mp4',
  'ACCEDE00031.mp4',
  'ACCEDE00032.mp4',
  'ACCEDE00033.mp4',
  'ACCEDE00034.mp4',
  'ACCEDE00035.mp4',
  'ACCEDE00036.mp4',
  'ACCEDE00037.mp4',
  'ACCEDE00038.mp4',
  'ACCEDE00039.mp4',
  'ACCEDE00040.mp4',
  'ACCEDE00041.mp4',
  'ACCEDE00042.mp4',
  'ACCEDE00043.mp4',
  'ACCEDE00044.mp4',
  'ACCEDE00045.mp4',
  'ACCEDE000

In [19]:
def getVaSequence(movie_name: str) -> list:
    ans = []
    for clip in movies[movie_name]:
        ans.append(getAffectiveValue(clip))
    return ans
getVaSequence('Dead_Man_Drinking.mp4')

[[3.0023439890900003, 1.3211960940799998],
 [3.35632469634, 1.4630537221000002],
 [3.3584827819300003, 1.55281387366],
 [3.3804810974299997, 1.31997605794],
 [3.38122382235, 2.3784131082099997],
 [2.4083920594799997, 1.6182376898800002],
 [3.34300320508, 2.1561033029799996],
 [1.64462153945, 3.46554854995],
 [3.37556216616, 1.8883937228],
 [3.3495881635000004, 2.66206988651],
 [3.37750914262, 2.70197853741],
 [2.7261733769700003, 2.18293366041],
 [2.63308601154, 2.5749861903099998],
 [3.0062146217900003, 2.3028031795599997],
 [3.2216992090900005, 1.6836344545200002],
 [1.53295961294, 4.243319058369999],
 [3.10817741171, 2.92069335528],
 [3.11106867855, 1.7251424691900004],
 [1.90751861591, 4.30274567015],
 [3.08692788186, 2.78103128825],
 [3.38257863922, 4.2515981930599995],
 [1.79873084262, 3.3301767597900005],
 [3.39052031769, 1.3626618558],
 [2.99876364036, 1.5669046600999998],
 [2.7585810889599998, 2.05831596815],
 [3.30135055132, 1.73292631181],
 [3.50015874075, 1.5480275663499998

In [ ]:
import matplotlib.path as mpath
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

%matplotlib inline

fig, ax = plt.subplots()
i = 0
print (f'movies count: {len(movies)}')
for movie_name in movies.keys():
    if i > 1:
        break
    i += 1
    x, y = zip(*getVaSequence(movie_name))
    line, = ax.plot(x, y, '-')

ax.grid()
# ax.axis([1,5,1,5])
ax.axis('equal')
ax.legend()
plt.xlabel('Arousal')
plt.ylabel('Valence')
plt.show()
    

# pytorch 读取视频数据

In [1]:
import torch
print(torch.__version__)
import torchvision
print(torchvision.__version__)
import numpy as np
import transform as T
normalize = T.Normalize(mean=[0.43216, 0.394666, 0.37645],
                            std=[0.22803, 0.22145, 0.216989])
transform_video = torchvision.transforms.Compose([
        T.ToFloatTensorInZeroOne(),
        T.Resize((128, 171)),
        T.RandomHorizontalFlip(),
        normalize,
        T.RandomCrop((112, 112))
    ])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

1.4.0
0.5.0


In [3]:
video_model = torchvision.models.video.r3d_18(pretrained=False, progress=True)
num_ftrs = video_model.fc.in_features
print(num_ftrs)
video_model.fc = torch.nn.Linear(num_ftrs, 2)
video_model = video_model.to(device)

512


In [28]:
import pandas as pd
import numpy as np
import torchaudio
print(torchaudio.__version__)
def normalize(tensor):
    # Subtract the mean, and scale to the interval [-1,1]
    tensor_minusmean = tensor - tensor.mean()
    return tensor_minusmean/tensor_minusmean.abs().max()
class LIRIS_ACCEDE(torch.utils.data.Dataset):
    def __init__(self, settype: str):
        super(LIRIS_ACCEDE).__init__()
        self.max_length = 200
        self.max_audio_length = 400000
        annotation_directory_path = '/root/yangsen-data/LIRIS-ACCEDE-annotations/LIRIS-ACCEDE-annotations/annotations/'
        sets_file_path = annotation_directory_path + 'ACCEDEsets.txt'
        sets_df = pd.read_csv(sets_file_path, sep='\t')
        if settype == 'train':
            set_id= 1
        elif settype == 'validation':
            set_id = 2
        elif settype == 'test':
            set_id = 0
        else:
            assert(false)
        self.videos_name = sets_df[sets_df['set'] == set_id]['name']
        self.data_path = '/root/yangsen-data/LIRIS-ACCEDE-data/data/'
        label_file = '/root/yangsen-data/LIRIS-ACCEDE-annotations/LIRIS-ACCEDE-annotations/annotations/ACCEDEranking.txt'
        self.label_df = pd.read_csv(label_file, sep='\t')
        self.mfcc_transformer = torchaudio.transforms.MFCC()
    def __len__(self):
        return len(self.videos_name)
    def __getitem__(self, idx):
        name = self.videos_name.iloc[idx]
        data = torchvision.io.read_video(self.data_path + name)
        sample_rate = data[2]['audio_fps']
        audio = data[1]
        downsample_rate=2000
        downsample_resample = torchaudio.transforms.Resample(
        sample_rate, downsample_rate, resampling_method='sinc_interpolation')
        audio = normalize(audio)
        if audio.shape[0] > 2:
            audio = audio[0:2,:]
        elif audio.shape[0] == 1:
            audio = audio.repeat(2, 1)
        if audio.shape[1] > self.max_audio_length:
            audio = audio[:,:self.max_audio_length]
        elif audio.shape[1] < self.max_audio_length:
            last_frame = audio[:,-1:]
            frames = last_frame.repeat(1,self.max_audio_length - audio.shape[1])
            audio = torch.cat([audio, frames],1)
        transformed_video = transform_video(data[0])
        if transformed_video.shape[1] > self.max_length:
            transformed_video = transformed_video[:,:self.max_length,:,:]
        elif self.max_length > transformed_video.shape[1]:
            last_frame = transformed_video[:,-1:,:,:]
            frames = last_frame.repeat(1,self.max_length-transformed_video.shape[1],1,1)
            transformed_video = torch.cat([transformed_video,frames],1)
#         print(transformed_video.shape)
            
        return {'name': name,
                'video': transformed_video,
                'audio': self.mfcc_transformer(audio),
                'label': np.array([self.label_df[self.label_df['name'] == name]['valenceValue'].iloc[0],
                          self.label_df[self.label_df['name'] == name]['arousalValue'].iloc[0]])
               }
    
trainset = LIRIS_ACCEDE(settype='train')
test_data_set = LIRIS_ACCEDE(settype='test')
validateset = LIRIS_ACCEDE(settype='validation')
train_data_set = torch.utils.data.ConcatDataset([trainset, validateset])
batch_size = 2
train_dataloader = torch.utils.data.DataLoader(train_data_set, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data_set, batch_size=batch_size)

0.4.0


In [26]:
trainset[0]

{'name': 'ACCEDE00000.mp4',
 'video': tensor([[[[-1.8952e+00, -1.8952e+00, -1.8952e+00,  ..., -1.8952e+00,
            -1.8952e+00, -1.8952e+00],
           [-1.8952e+00, -1.8952e+00, -1.8952e+00,  ..., -1.8952e+00,
            -1.8952e+00, -1.8952e+00],
           [-1.8952e+00, -1.8952e+00, -1.8952e+00,  ..., -1.8952e+00,
            -1.8952e+00, -1.8952e+00],
           ...,
           [-1.8952e+00, -1.8952e+00, -1.8952e+00,  ..., -1.8952e+00,
            -1.8952e+00, -1.8952e+00],
           [-1.8952e+00, -1.8952e+00, -1.8952e+00,  ..., -1.8952e+00,
            -1.8952e+00, -1.8952e+00],
           [-1.8952e+00, -1.8952e+00, -1.8952e+00,  ..., -1.8952e+00,
            -1.8952e+00, -1.8952e+00]],
 
          [[-1.8952e+00, -1.8952e+00, -1.8952e+00,  ..., -1.8952e+00,
            -1.8952e+00, -1.8952e+00],
           [-1.8952e+00, -1.8952e+00, -1.8952e+00,  ..., -1.8952e+00,
            -1.8952e+00, -1.8952e+00],
           [-1.8952e+00, -1.8952e+00, -1.8952e+00,  ..., -1.8952e+00,
  

In [34]:
trainset[2]['audio'].shape

torch.Size([2, 40, 2001])

In [4]:
audio_model = torchvision.models.resnet18(pretrained=True)
num_ftrs = audio_model.fc.in_features
print(num_ftrs)
audio_model.fc = torch.nn.Linear(num_ftrs, 2)
audio_model.conv1 = torch.nn.Conv2d(2,64,(2,10))
audio_model = audio_model.to(device)

512


In [4]:
def trainProcess(model, modal='video'):
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    for epoch in range(3):
        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            video_inputs = data[modal].to(device)
            labels = torch.tensor(data['label']).float().to(device)
            
            optimizer.zero_grad()

            outputs = model(video_inputs)
#             print(outputs.shape, labels.shape)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 5 == 4:
                print('[%d. %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 5))
                running_loss = 0.0

In [ ]:
import time
time1 = time.time()
trainProcess(audio_model, modal='audio')
time2 = time.time()
print (f'audio train: {time2 - time1}s')

/root/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


[1.     5] loss: 0.209
[1.    10] loss: 0.474
[1.    15] loss: 0.734
[1.    20] loss: 0.553
[1.    25] loss: 0.357
[1.    30] loss: 1.087
[1.    35] loss: 0.491
[1.    40] loss: 0.761
[1.    45] loss: 0.798
[1.    50] loss: 0.390
[1.    55] loss: 0.974
[1.    60] loss: 0.900
[1.    65] loss: 1.144
[1.    70] loss: 1.008
[1.    75] loss: 0.320
[1.    80] loss: 0.323
[1.    85] loss: 0.236
[1.    90] loss: 0.398
[1.    95] loss: 0.484
[1.   100] loss: 0.363
[1.   105] loss: 0.601
[1.   110] loss: 0.787
[1.   115] loss: 1.016
[1.   120] loss: 0.546
[1.   125] loss: 0.450
[1.   130] loss: 0.350
[1.   135] loss: 0.045
[1.   140] loss: 0.593
[1.   145] loss: 0.835
[1.   150] loss: 0.662
[1.   155] loss: 0.364
[1.   160] loss: 0.444
[1.   165] loss: 0.773
[1.   170] loss: 0.898
[1.   175] loss: 0.532
[1.   180] loss: 0.552
[1.   185] loss: 0.391
[1.   190] loss: 1.144
[1.   195] loss: 0.637
[1.   200] loss: 0.599
[1.   205] loss: 1.181
[1.   210] loss: 0.896
[1.   215] loss: 1.055
[1.   220] 

In [ ]:
trainProcess(video_model, 'video')

In [16]:
torch.save(video_model.state_dict(), 'video_model.model')

In [3]:
video_model = torchvision.models.video.r3d_18(pretrained=False, progress=True)
num_ftrs = video_model.fc.in_features
video_model.fc = torch.nn.Linear(num_ftrs, 2)
video_model.load_state_dict(torch.load('video_model.model'))
video_model.eval()
video_model = video_model.to(device)

In [7]:
def evalProcess(model):
    criterion = torch.nn.MSELoss()
    arousal_loss_test = 0.0
    valence_loss_test = 0.0
    mse_list = []
    for i, data in enumerate(test_dataloader, 0):
        # get the inputs
        inputs = data['video']
        valence = data['label'][:, 0:1]
        arousal = data['label'][:, 1:2]
        ground_truth = data['label']
        inputs, valence, arousal, ground_truth = inputs.to(device), valence.to(device), arousal.to(device), ground_truth.to(device)
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            valence_loss = criterion(outputs[:, 0:1], valence)
            arousal_loss = criterion(outputs[:, 1:2], arousal)
            arousal_loss_test += arousal_loss.item()
            valence_loss_test += valence_loss.item()

    print(f'arousal mse average: {arousal_loss_test / len(test_dataloader)}')
    print(f'valence mse average: {valence_loss_test / len(test_dataloader)}')
    mse_list.append((arousal_loss_test / len(test_dataloader), valence_loss_test / len(test_dataloader)))

import time
t1 = time.time()
evalProcess(video_model)
t2 = time.time()
print(t2 - t1)

arousal mse average: 1.4676290782621924
valence mse average: 0.4838273013567208
11344.869487285614
